# Tidal Basins

## First import some necessary packages

In [ ]:
import pathlib
import sys
from random import shuffle, uniform

import IPython
import ipywidgets as widgets
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator

import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
from PIL import Image

import coastal_dynamics as cd

pn.extension()

In [5]:
# # Read questions from cloud storage
questions = cd.read_questions(
    "./8_tidal_basins.json"
    # "az://coastal-dynamics/questions/8_tidal_basins_hashed.json",
    # storage_options={"account_name": "coclico"},
)

In [1]:
%run initialize/init_8.ipynb

Packages succesfully loaded


Questions succesfully loaded


Before you is the Jupiter Notebook of week 8. This notebook consists of 4 parts:

1. Escoffier curve
2. Van de Kreeke and Robaczewska
3. Equilibrium concentration versus lag
4. Interventions


The student will practice with these topics through questions and interactive figures.

## Part 1: Escoffier curve

In chapter 9 of the [Coastal Dynamics Open Textbook](https://textbooks.open.tudelft.nl/textbooks/catalog/view/37/92/383-1), we learned about tidal inlets and their stability. Tidal inlets are very dynamic and their stability depends on a lot of factors. Escoffier was the first to study the stability of tidal inlets. More specifically, he studied the cross-sectional area of tidal inlets as they change throughout the tidal cycle. From his studies, Escoffier developed the now well-known Escoffier curve. In this notebook we will use an interactive approach to better understand the Escoffier curve. Afterwards you are tasked to answer some questions to verify your knowledge on the topic.

Before starting this notebook, make sure you followed the lectures on chapter 9 (or read the slides) and read section 9.5.1 of the book.

Escoffier’s curve is a so-called closure curve and describes the relationship between maximum channel velocity $u_e$ and the parameter $X$, which is primarily, but not solely, a function of the channel cross-section. If we consider a sinusoidal tidal velocity signal:

$$
\begin{aligned}
u_e = \hat{u}_e = \frac{\pi P}{A_e T}
\end{aligned}
$$

Where $\hat{u}_e$ is the tidal signal amplitude, $P$ the tidal prism, $A_e$ the channel cross-section and $T$ the tidal period (see Intermezzo 9.4 of the book, equation 9.5).


The process that leads to the Escoffier curve is explained in the book. In this notebook we provide a short visualisation, see the interactive plot below. We start with an imaginary channel cross-section that is very small, close to point A, such that the tidal difference in the estuary is smaller than the tidal range. Increasing the cross-section ($A_e$) results in an increase of the tidal prism ($P$) so large that $u_e$ increases too (recall eq. 9.5). At some point the tidal difference in the estuary is equal to the tidal range and we reach the peak of the closure curve. A larger cross-section now reduces $u_e$ as $P$ remains constant (again, recall eq. 9.5).

In [2]:
# Load the images
images = [
    Image.open('figures/04_Escoffier_interactive_1.png'),
    Image.open('figures/04_Escoffier_interactive_2.png'),
    Image.open('figures/04_Escoffier_interactive_3.png'),
    Image.open('figures/04_Escoffier_interactive_4.png'),
    Image.open('figures/04_Escoffier_interactive_5.png'),
    Image.open('figures/04_Escoffier_interactive_6.png'),
    Image.open('figures/04_Escoffier_interactive_7.png')
]

# Create the slider widget
slider = widgets.IntSlider(min=0, max=len(images)-1, step=1, value=0)

# Display the current image
image_widget = widgets.Image(value=images[slider.value]._repr_png_(), format='png', width='95%')

# Define the update function
def update_image(change):
    image_widget.value = images[change.new]._repr_png_()

# Attach the update function to the slider
slider.observe(update_image, names='value')

# Display the widgets
display(slider)
display(image_widget)

IntSlider(value=0, max=6)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xef\x00\x00\x02\x15\x08\x02\x00\x00\x00\xc7\xd3\…

The next step is to determine an equilibrium channel velocity $u_{eq}$ below which no erosion of the channel occurs. This velocity is only slightly dependent on the cross-section and can be approximated as just a function of sediment size. Larger sediment size leads to a larger $u_{eq}$ and vice versa. The closure curve and a value for $u_{eq}$ leads to the well-known Escoffier curve as depicted in Figure 9.22 in the book, as depicted below:

![04_Ch9_Escoffier_type_1.png](./figures/04_Ch9_Escoffier_type_1.png)


### Questions

Now that you know how an Escoffier curve is created and some of the physical processes behind it, it's time to test your understanding. Try to answer the questions below and give your answers in the corresponding codeblock.

#### Channel stability

Consider the Escoffier curve below with points A, B, C, D and E. What happens to the channel cross-section ($X$) at locations 1, 2, 3, 4 and 5? Where does it move to, point A, B, C, D, E or neither (N)? Give your answers below and run the codeblock to validate your answers and view your feedback.

![04_Ch9_Escoffier_stability_1](./figures/04_Ch9_Escoffier_stability_1.png)

In [6]:
q1 = cd.QuestionFactory(questions["Q8-1"]).serve()
q2 = cd.QuestionFactory(questions["Q8-2"]).serve()
q3 = cd.QuestionFactory(questions["Q8-3"]).serve()
q4 = cd.QuestionFactory(questions["Q8-4"]).serve()
q5 = cd.QuestionFactory(questions["Q8-5"]).serve()


pn.Column(q1, q2, q3, q4, q5)

KeyError: 'Q8-1'

#### Escoffier curve "types"

Below three Escoffier curves are shown. So far we have only considered the scenario where Point C lies above $u_{eq}$ (scenario I). However, two other scenarios can also exist: II where point C coincides with $u_{eq}$ and III where point C is below $u_{eq}$. Similar to the previous question, what happens now at locations 1 and 2 for scenarios II and III?

![04_Ch9_Escoffier_type_1.png](./figures/04_Ch9_Escoffier_type_1.png)

![04_Ch9_Escoffier_type_2.png](./figures/04_Ch9_Escoffier_type_2.png)

![04_Ch9_Escoffier_type_3.png](./figures/04_Ch9_Escoffier_type_3.png)

Put your answers here:

In [ ]:
question_data16 = {
        "name": "Q1-6",
        "question": "What happens now at point 1 for scenario II?",
        "options": {
            "a": "A",
            "b": "B",
            "c": "C",
            "d": "D",
            "e": "E",
            "f": "N"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data17 = {
        "name": "Q1-7",
        "question": "What happens now at point 2 for scenario II?",
        "options": {
            "a": "A",
            "b": "B",
            "c": "C",
            "d": "D",
            "e": "E",
            "f": "N"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data18 = {
        "name": "Q1-8",
        "question": "What happens now at point 3 for scenario II?",
        "options": {
            "a": "A",
            "b": "B",
            "c": "C",
            "d": "D",
            "e": "E",
            "f": "N"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data19 = {
        "name": "Q1-9",
        "question": "What happens now at point 1 for scenario III?",
        "options": {
            "a": "A",
            "b": "B",
            "c": "C",
            "d": "D",
            "e": "E",
            "f": "N"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data110 = {
        "name": "Q1-10",
        "question": "What happens now at point 2 for scenario III?",
        "options": {
            "a": "A",
            "b": "B",
            "c": "C",
            "d": "D",
            "e": "E",
            "f": "N"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data111 = {
        "name": "Q1-11",
        "question": "What happens now at point 3 for scenario III?",
        "options": {
            "a": "A",
            "b": "B",
            "c": "C",
            "d": "D",
            "e": "E",
            "f": "N"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }


pn.Column(MC(question_data16), 
          MC(question_data17), 
          MC(question_data18), 
          MC(question_data19),
          MC(question_data110),
          MC(question_data111))

# ansII_1  = 'A'  # Your answer for point 1 scenario II
# ansII_2  = 'A'  # Your answer for point 2 scenario II
# ansII_3  = 'A'  # Your answer for point 3 scenario II

# ansIII_1 = 'A'  # Your answer for point 1 scenario III
# ansIII_2 = 'A'  # Your answer for point 2 scenario III
# ansIII_3 = 'A'  # Your answer for point 3 scenario III

# #The function below returns your result
# Check2(ansII_1,ansII_2,ansII_3,ansIII_1,ansIII_2,ansIII_3)

#### Changes to the Escoffier curve

So far we have considered the typical Escoffier curve and asked you some questions on what happens with the tidal inlet. You have mastered working with the Escoffier curve, but are you also able to answer the questions below?

In [ ]:
question_data112 = {
        "name": "Q1-12",
        "question": "What would happen if X becomes really large?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data113 = {
        "name": "Q1-13",
        "question": "Can X go to infinity?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data114 = {
        "name": "Q1-14",
        "question": "How can you modify the Escoffier curve through engineering?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answers": ["a", "b"],
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data115 = {
        "name": "Q1-15",
        "question": "Can the closure curve vary in time?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }


pn.Column(MC(question_data112), 
          MC(question_data113), 
          MS(question_data114), 
          MC(question_data115))

## Part 2: Van de Kreeke and Robaczewska
(STUART introduce students to topic, show formula from Van de Kreeke and Robaczewska)

$u(t) = u_0 + \hat{u}_{M2} \cos(\omega_{M2} t) + \hat{u}_{M4} \cos(\omega_{M4} t - \phi_{M4-2}) + \hat{u}_{M6} \cos(\omega_{M6} t - \phi_{M6-2})$

In [ ]:
def plot_u(u0, um2, um4, um6, phi42, phi62):
    fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (9,6), sharex=True, sharey = False)
    fig.subplots_adjust(hspace=0.05)
    fig.subplots_adjust(wspace=0.04)

    T2 = 12*3600 + 25 * 60
    T4 = (12*3600 + 25 * 60) / 2
    T6 = (12*3600 + 25 * 60) / 3
    
    omega2 = 2*np.pi/T2
    omega4 = 2*np.pi/T4
    omega6 = 2*np.pi/T6
                        
    t = np.linspace(0, 24*3600, 200)

    M2 = um2 * np.cos(omega2 * t)
    M4 = um4 * np.cos(omega4 * t - phi42)
    M6 = um6 * np.cos(omega6 * t - phi62)
    
    u = u0 + M2 + M4 + M6

    axs.plot(t, u0*np.ones(t.shape), color='C0', label='u0', alpha=0.25, linestyle='--')
    axs.plot(t, M2, color='C1', label='M2 component', alpha=0.25, linestyle='--')
    axs.plot(t, M4, color='C2', label='M4 component', alpha=0.25, linestyle='--')
    axs.plot(t, M6, color='C3', label='M6 component', alpha=0.25, linestyle='--')
    axs.plot(t, u, color='k', label='u total')
    
    axs.legend(loc='upper right')

    axs.set_xlabel('time [h]')
    axs.set_xticks([0, 86400/4, 86400/2, 86400*3/4, 86400], [0, 6, 12, 18, 24])
    axs.set_ylabel('u [m/s]')

def slider_u():
    # Create interactive widgets, requires IPY Widgets, widgets from panel do not work
    u0 = ipw.FloatSlider(value=1, min=-5, max=5, step=0.01, description=r'u_0 [m/s]')
    um2 = ipw.FloatSlider(value=0.5, min=0, max=5, step=0.01, description="u_M2 [m/s]")
    um4 = ipw.FloatSlider(value=0.2, min=0, max=5, step=0.01, description="u_M4 [m/s]")
    um6 = ipw.FloatSlider(value=0.1, min=0, max=5, step=0.01, description="u_M6 [m/s]")
    phi42 = ipw.FloatSlider(value=0, min=0, max=2*np.pi, step=0.01, description="phi_42 [rad]")
    phi62 = ipw.FloatSlider(value=0, min=0, max=2*np.pi, step=0.01, description="phi_62 [rad]")

    # Use the interactive function to update the plot
    Plot = interact(plot_u, u0=u0, um2=um2, um4=um4, um6=um6, phi42=phi42, phi62=phi62);
    display(Plot);

slider_u();

It is assumed that the sediment transport depends on the third power of the velocity signal: $S \approx c u^3 \sim u^3$.
Let's plot the third power of the velocity:

In [ ]:
def plot_u3(u0, um2, um4, um6, phi42, phi62):
    fig, axs = plt.subplots(nrows = 1, ncols = 1, figsize = (9,6), sharex=True, sharey = False)
    fig.subplots_adjust(hspace=0.05)
    fig.subplots_adjust(wspace=0.04)

    T2 = 12*3600 + 25 * 60
    T4 = (12*3600 + 25 * 60) / 2
    T6 = (12*3600 + 25 * 60) / 3
    
    omega2 = 2*np.pi/T2
    omega4 = 2*np.pi/T4
    omega6 = 2*np.pi/T6
                        
    t = np.linspace(0, 24*3600, 200)

    M2 = um2 * np.cos(omega2 * t)
    M4 = um4 * np.cos(omega4 * t - phi42)
    M6 = um6 * np.cos(omega6 * t - phi62)
    
    u = u0 + M2 + M4 + M6
    u3 = u**3

    axs.plot(t, u0*np.ones(t.shape), color='C0', label='u0', alpha=0.25, linestyle='--')
    axs.plot(t, M2, color='C1', label='M2 component', alpha=0.25, linestyle='--')
    axs.plot(t, M4, color='C2', label='M4 component', alpha=0.25, linestyle='--')
    axs.plot(t, M6, color='C3', label='M6 component', alpha=0.25, linestyle='--')
    axs.plot(t, u, color='grey', label='u', alpha=0.25)
    axs.plot(t, u3, color='k', label='$u^3$')
    
    axs.legend(loc='upper right')

    axs.set_xlabel('time [h]')
    axs.set_xticks([0, 86400/4, 86400/2, 86400*3/4, 86400], [0, 6, 12, 18, 24])
    axs.set_ylabel('u$^3$ [$m^3/s^3$]')

def slider_u3():
    # Create interactive widgets, requires IPY Widgets, widgets from panel do not work
    u0 = ipw.FloatSlider(value=1, min=0, max=5, step=0.01, description=r'u_0 [m/s]')
    um2 = ipw.FloatSlider(value=0.5, min=0, max=5, step=0.01, description="u_M2 [m/s]")
    um4 = ipw.FloatSlider(value=0.2, min=0, max=5, step=0.01, description="u_M4 [m/s]")
    um6 = ipw.FloatSlider(value=0.1, min=0, max=5, step=0.01, description="u_M6 [m/s]")
    phi42 = ipw.FloatSlider(value=0, min=0, max=2*np.pi, step=0.01, description="phi_42 [rad]")
    phi62 = ipw.FloatSlider(value=0, min=0, max=2*np.pi, step=0.01, description="phi_62 [rad]")

    # Use the interactive function to update the plot
    Plot = interact(plot_u3, u0=u0, um2=um2, um4=um4, um6=um6, phi42=phi42, phi62=phi62);
    display(Plot)

slider_u3();

Have a look at figures 9.28 - 9.31 in the book. 

STUART ask questions (related to reproducing the plots).

In [ ]:
question_data21 = {
        "name": "Q2-1",
        "question": "...",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data22 = {
        "name": "Q2-2",
        "question": "...",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data23 = {
        "name": "Q2-3",
        "question": "...",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data24 = {
        "name": "Q2-4",
        "question": "...",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data21), 
          MC(question_data22), 
          MC(question_data23), 
          MC(question_data24))

The table below contains velocity and phase data for an (imaginary) estuary.

| *Amplitude [cm/s]* |     | *Phase [degrees]* |     |
| :----------------- | :-- | :---------------- | :-- |
| M0                 | -5  | phi42             | 240 |
| M2                 | 80  | phi62             | 200 |
| M4                 | 15  |                   |     |
| M6                 | 25  |                   |     |  |  |pute 

Cototal thel transp tota as a fun of tction. Use all harmonic components for this. Also compute the total net average transport $\frac{\langle S \rangle}{c \hat{u}^3_{M2}}$ d toll a M  an and M4/M2/M0,  contributions. Use a value of $10^{-4}$ for {c}.

Question: why do we not compute the M6/M2 tran?ions.

In [ ]:
t = np.linspace(0, 24*3600, 250)

# Correct the formula for the total transport here
total_transport = np.zeros(t.shape)

In [ ]:
# Plot total transport as a function of t

def Q2_5(total_transport):
    t = np.linspace(0, 2*24*3600, 250)

    try:
        total_transport
    except UnboundLocalError:
        total_transport = np.zeros(t.shape)

    c = 10**-4

    u0 = -5/100
    um2 = 80/100
    um4 = 15/100
    um6 = 25/100

    phi42 = 240 / 360 * (2*np.pi)
    phi62 = 200 / 360 * (2*np.pi)

    T2 = 12*3600 + 25 * 60
    T4 = (12*3600 + 25 * 60) / 2
    T6 = (12*3600 + 25 * 60) / 3
    
    omega2 = 2*np.pi/T2
    omega4 = 2*np.pi/T4
    omega6 = 2*np.pi/T6
                        
    M2 = um2 * np.cos(omega2 * t)
    M4 = um4 * np.cos(omega4 * t - phi42)
    M6 = um6 * np.cos(omega6 * t - phi62)
    
    S = c*(u0 + M2 + M4 + M6)**3

    df = pd.DataFrame({'t':t, 'Correct transport':S, 'Student transport':total_transport})

    plot = df.hvplot.line(x='t', y=['Correct transport', 'Student transport'])

    return plot

Q2_5(total_transport)

In [ ]:
question_data25 = {
        "name": "Q2-5",
        "question": "What is the total net average dimensionless transport due to the M0/M2 contribution? Answer with a precision of three decimal spaces.",
        "answer": round(3/2 * -5/80, 3),
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"},
        "precision":3  # number of decimal spaces
    }

question_data26 = {
        "name": "Q2-6",
        "question": "What is the total net average dimensionless transport due to the M2/M4 contribution? Answer with a precision of three decimal spaces.",
        "answer": round(3/4 * 15/80 * np.cos(240 / 360 * (2*np.pi)), 3),
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"},
        "precision":3  # number of decimal spaces
    }

question_data27 = {
        "name": "Q2-7",
        "question": "Why do we not compute the M6/M2 transport?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data28 = {
        "name": "Q2-8",
        "question": "What does (some contribution of an overtide) mean?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data29 = {
        "name": "Q2-9",
        "question": "Why is the transport high for (some specific overtide)?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data210 = {
        "name": "Q2-10",
        "question": "In the book some assumptions were made about neglecting some of the components. Which components are neglected in the book?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answers": ["a", "b"],
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data211 = {
        "name": "Q2-11",
        "question": "In the book some assumptions were made about neglecting some of the components. Which components can be neglected with respect to the total transport?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answers": ["a", "b"],
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data212 = {
        "name": "Q2-12",
        "question": "Which components are most important? Compare with equation (...) from the book.",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answers": ["a","b"],
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
}
  


In [ ]:
pn.Column(num(question_data25), 
          num(question_data26), 
          MC(question_data27), 
          MC(question_data28),
          MC(question_data29), 
          MS(question_data210), 
          MS(question_data211),
          MS(question_data212))

## Part 3: Equilibrium concentration versus lag
STUART Introduce topic here.

The table below gives velocity amplitudes and phases of the first and second harmonic of the tide.

| *Amplitude [cm/s]* |     | *Phase [degrees]* |     |
| :----------------- | :-- | :---------------- | :-- |
| M0                 | -20 | phi42             | 180 |
| M2                 | 40  |                   |     |
| M4                 | 10  |                   |     |

Complete the code below to plot the tidal velocities.

In [ ]:
t = np.linspace(0, 24*3600, 250)

# Correct the formula for the tidal velocities below
u = np.zeros(t.shape)

In [ ]:
def Q3_1(u):
    t = np.linspace(0, 2*24*3600, 250)

    try:
        u
    except UnboundLocalError:
        u = np.zeros(t.shape)

    u0 = -20/100
    um2 = 40/100
    um4 = 10/100

    phi42 = 180 / 360 * (2*np.pi)

    T2 = 12*3600 + 25 * 60
    T4 = (12*3600 + 25 * 60) / 2
    
    omega2 = 2*np.pi/T2
    omega4 = 2*np.pi/T4
                        
    M2 = um2 * np.cos(omega2 * t)
    M4 = um4 * np.cos(omega4 * t - phi42)
    
    u_correct = u0 + M2 + M4

    df = pd.DataFrame({'t':t, 'Correct velocity':u_correct, 'Student velocity':u})

    plot = df.hvplot.line(x='t', y=['Correct velocity', 'Student velocity'])

    return plot
    
Q3_1(u)

Use equation 9.29 to plot the equillibrium concentration. Use values of ... and ... for $\beta$ and $n$ respectively.

In [ ]:
beta = 10**-4
n = 5

# Correct the formula for the equillibrium concentration below
c_eq = np.zeros(t.shape)

In [ ]:
### Hidden codeblock


def Q3_2(c_eq, beta, n):
    t = np.linspace(0, 2*24*3600, 250)

    u0 = -20/100
    um2 = 40/100
    um4 = 10/100

    phi42 = 180 / 360 * (2*np.pi)

    T2 = 12*3600 + 25 * 60
    T4 = (12*3600 + 25 * 60) / 2
    
    omega2 = 2*np.pi/T2
    omega4 = 2*np.pi/T4
                        
    M2 = um2 * np.cos(omega2 * t)
    M4 = um4 * np.cos(omega4 * t - phi42)
    
    u_correct = u0 + M2 + M4

    c_eq_correct = beta * np.abs(u_correct)**(n-1)

    df = pd.DataFrame({'t':t, 'Tidal velocity * 10^-5':u_correct * 10**-5, 'Correct equillibrium concentration':c_eq_correct, 'Student equillibrium concentration':c_eq})

    plot = df.hvplot.line(x='t', y=['Correct equillibrium concentration', 'Student equillibrium concentration', 'Tidal velocity * 10^-5'])

    return plot

Q3_2(c_eq, beta, n)

Imagine we increase the sediment size. Answer the questions below.

In [ ]:
question_data33 = {
        "name": "Q3-3",
        "question": "What happens to sediment response lag if we increase the sediment size?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data34 = {
        "name": "Q3-4",
        "question": "What happens to sediment transport magnitude if we increase the sediment size?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data33),
          MC(question_data34)
         )

Looking at equation 9.28 in the book, we can see that the equation for the the actual concentration contains a time derivative. Therefore, in order to obtain a solution for the concentration, a discretization is required. Use the following discretization (Forward Euler) for quantity u in order to model the sediment concentration:

$\frac{D u}{D t} \approx \frac{u_{n+1} - u_{n}}{\Delta t}$. 

The timestep $\Delta t$ is already given in the code. Use a relaxation time scale of 12000 seconds. Use your solution for the equillibrium concentration from earlier.

Complete the code below to plot the concentration as a function of time.

In [ ]:
t = np.linspace(0, 2*24*3600, 1000)
dt = t[1]-t[0]

c = np.zeros(t.shape)

for n in range(len(c)-1):

    # Add your code here
    ...



In [ ]:
def Q3_5(student_c):

    t = np.linspace(0, 2*24*3600, 1000)
    
    u0 = -20/100
    um2 = 40/100
    um4 = 10/100
    
    phi42 = 180 / 360 * (2*np.pi)
    
    T2 = 12*3600 + 25 * 60
    T4 = (12*3600 + 25 * 60) / 2
    
    omega2 = 2*np.pi/T2
    omega4 = 2*np.pi/T4
                        
    M2 = um2 * np.cos(omega2 * t)
    M4 = um4 * np.cos(omega4 * t - phi42)
    
    u_correct = u0 + M2 + M4
    
    beta = 10**-4
    n = 5
    
    c_eq_correct = beta * np.abs(u_correct)**(n-1)
    
    c_correct = np.zeros(c_eq_correct.shape)
    
    dt = t[1]-t[0]
    Tsed = 12000
    
    for n in range(len(c)-1):
        c_correct[n+1] = c_correct[n] + dt * 1/Tsed * (c_eq_correct[n]-c_correct[n])
    
    # print(c)
    
    # plt.plot(t, u_correct)

    df = pd.DataFrame({"t":t, "Correct concentration":c_correct, "Student concentration": student_c, "Equillibrium concentration": c_eq_correct})
    
    plot = df.hvplot.line(x='t', y=['Correct concentration', 'Student concentration', 'Equillibrium concentration'])

    return plot


Q3_5(c)

Reflect on the plots you created with the questions below.

In [ ]:
# Sediment size, sediment response, lag effect, sediment transport magnitude.

question_data36 = {
        "name": "Q3-6",
        "question": "STUART ADD QUESTION ON SEDIMENT SIZE",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data37 = {
        "name": "Q3-7",
        "question": "STUART ADD QUESTION ON SEDIMENT RESPONSE",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data38 = {
        "name": "Q3-8",
        "question": "STUART ADD QUESTION ON LAG EFFECT",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data39 = {
        "name": "Q3-9",
        "question": "STUART ADD QUESTION ON SEDIMENT TRANSPORT MAGNITUDE",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data36),
          MC(question_data37),
          MC(question_data38),
          MC(question_data39)
         )

If we remove the phase difference, i.e. $\phi=0$, the sediment response is expected to change. Adjust the code above to reflect this. Run your code again. What happens? Compare to figure 9.32 of the textbook.

In [ ]:
# Add questions here...

question_data310 = {
        "name": "Q3-10",
        "question": "STUART ADD REFLECTIVE QUESTION",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data311 = {
        "name": "Q3-11",
        "question": "STUART ADD REFLECTIVE QUESTION",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
}

pn.Column(MC(question_data310),
          MC(question_data311)
         )

## Part 4: Interventions
STUART INTRODUCE TOPIC HERE.

Figures 9.35 and 9.36 in the book show the effect of two different closures. The figure (without interventions) is shown also below. We are going to try to reproduce the interventions made in 9.35. 

In [ ]:
# Show figures 9.35 and 9.36.
def part4_intro():
    P = np.logspace(5, 11, 100)
    Cv = 65 * 10**-6
    Cod = 65.7 * 10**-4
    
    Vod = Cod * P**1.23
    Vc  = Cv  * P**1.5
    
    df = pd.DataFrame({"P * 10^6":P/10**6, "V_od * 10^6":Vod/10**6, "V_c * 10^6":Vc/10**6})
    
    plot = df.hvplot.line(x='P * 10^6', y=['V_od * 10^6', 'V_c * 10^6'], grid=True, loglog=True, xlim=(10, 5000), ylim=(2, 5000))
    
    return plot

part4_intro()

Table 9.6 is (partly) given below. All variables in $10^6$ m$^3$.

|                        | closure 1 |
| :--------------------- | :-------- |
| Prism before           | 600       |
| Prism after            | 300       |
| Difference in Vc <br>  | 300       |   |   |   |
   |

Complete the code below to calculate the $V_{od}$ and $V_c$ for closure 1. Also calculate the sand requirements for the channel $a$ and the surplus of sand in the outer delta $b$. Use values of $65 \times 10^{-6}$ and $65.7 \times 10^{-4}$ for $C_V$ and $C_{od}$ respectively. Answer the questions below.

In [ ]:
V_c_before  = ...
V_c_after   = ...

V_od_before = ...
V_od_after  = ...


a = ...
b = ...

In [ ]:
# correct answers
V_c_before  = 65*10**-6 * (600*10**6)**1.5
V_c_after   = 65*10**-6 * (300*10**6)**1.5

V_od_before = 65.7*10**-4 * (600*10**6)**1.23
V_od_after  = 65.7*10**-4 * (300*10**6)**1.23


a = 65*10**-6 * (600*10**6)**1.5 - 65*10**-6 * (300*10**6)**1.5 - 300*10**6
b = 65.7*10**-4 * (600*10**6)**1.23 - 65.7*10**-4 * (300*10**6)**1.23

In [ ]:
def plot_fig935(P_before, P_after, dV_c,
                axes_type='loglog',
                range=[[10, 5000], [2, 5000]],
                coefficients=[65 * 10**-6, 65.7 * 10**-4],
                powers=[1.5, 1.23],
                title='Correct plot'
               ):
    
    P = np.logspace(2, 20, 100)

    power_c, power_od  = powers[0], powers[1]
    Cv, Cod = coefficients[0], coefficients[1]

    xmin, xmax = range[0]
    ymin, ymax = range[1]
    
    Vc  = Cv  * P**power_c
    Vod = Cod * P**power_od

    if axes_type=='loglog':
        loglog=True
    elif axes_type=='linear':
        loglog=False
    else:
        raise ValueError('Use either loglog or linear for the axes_type')
    
    df = pd.DataFrame({"P * 10^6":P/10**6, "V_od * 10^6":Vod/10**6, "V_c * 10^6":Vc/10**6})
    
    plot = df.hvplot.line(x='P * 10^6', y=['V_od * 10^6', 'V_c * 10^6'], grid=True, loglog=loglog, xlim=(xmin, xmax), ylim=(ymin, ymax), color=['blue', 'red'], title=title)
    
    dP = P_before - P_after
    
    V_c_before_correct = Cv * P_before**power_c
    V_c_new_correct = V_c_before_correct - dV_c
    V_c_after_correct = Cv * P_after**power_c
    
    V_od_before_correct = Cod * P_before**power_od
    V_od_after_correct = Cod * P_after**power_od
    
    a_correct = V_c_before_correct - dV_c - V_c_after_correct
    b_correct = V_od_before_correct - V_od_after_correct
    
    dP_curve = hv.Curve([(P_before/10**6, V_c_before_correct/10**6), (P_after/10**6, V_c_before_correct/10**6)]).opts(color='black', apply_ranges=True, line_dash='solid', line_width=1) * \
               hv.Text((P_before+P_after)/2/10**6, V_c_before_correct/10**6 + 300, 'dP', fontsize=9)
    
    dV_c_curve = hv.Curve([(P_after/10**6, V_c_before_correct/10**6), (P_after/10**6, (V_c_before_correct-dV_c)/10**6)]).opts(color='black', apply_ranges=True, line_dash='solid', line_width=1) * \
                 hv.Text((P_after/10**6)-50, (V_c_before_correct+V_c_before_correct-dV_c)/2/10**6, 'dVc', fontsize=9)
    
    connection_curve = hv.Curve([[P_before/10**6, V_c_before_correct/10**6], [(P_before-dP)/10**6, (V_c_before_correct-dV_c)/10**6]]).opts(color='black', line_dash='dashed')
    
    a_curves = hv.Curve([[10**-5, (V_c_before_correct-dV_c)/10**6], [P_after/10**6, (V_c_before_correct-dV_c)/10**6]]).opts(color='red', line_dash='dashed') * \
                   hv.Curve([[10**-5, V_c_after_correct/10**6], [P_after/10**6, V_c_after_correct/10**6]]).opts(color='red', line_dash='dashed') * \
                   hv.Curve([[10**2+10, V_c_after_correct/10**6], [10**2+10, (V_c_before_correct-dV_c)/10**6]]).opts(color='black', line_width=1) * \
                   hv.Text(10**2, (V_c_after_correct/10**6+(V_c_before_correct-dV_c)/10**6)/2, text='a', fontsize=9)
    
    b_curves = hv.Curve([[P_after/10**6, V_od_after_correct/10**6], [10**6, V_od_after_correct/10**6]]).opts(color='blue', line_dash='dashed') * \
                   hv.Curve([[P_before/10**6, V_od_before_correct/10**6], [10**6, V_od_before_correct/10**6]]).opts(color='blue', line_dash='dashed') * \
                   hv.Curve([[10**3, V_od_after_correct/10**6], [10**3, (V_od_before_correct)/10**6]]).opts(color='black', line_width=1) * \
                   hv.Text(10**3 + 70, (V_od_after_correct/10**6+V_od_before_correct/10**6)/2, text='b', fontsize=9)
    
    points_old = hv.Points([[P_before/10**6, V_c_before_correct/10**6], [P_before/10**6, V_od_before_correct/10**6]]).opts(size=10, marker='x', color='white', line_color='black')
    points_bet = hv.Points([[P_after/10**6, V_c_new_correct/10**6]]).opts(size=10, marker='circle', color='black', line_color='black')
    points_new = hv.Points([[P_after/10**6, V_c_after_correct/10**6], [P_after/10**6, V_od_after_correct/10**6]]).opts(size=10, marker='circle', color='white', line_color='black')
    
    points = points_old * points_bet * points_new
    
    plot_correct = plot * dP_curve * dV_c_curve * connection_curve * a_curves * b_curves * points

    return plot_correct, V_c_before_correct, V_c_after_correct, V_od_before_correct, V_od_after_correct, a_correct, b_correct

In [ ]:
def student_plot_fig935(P_before, P_after, dV_c,
                        V_c_before,
                        V_c_after,
                        V_od_before,
                        V_od_after,
                        a,
                        b,
                        Cv=65 * 10**-6, 
                        Cod= 65.7 * 10**-4):

        P = np.logspace(2, 20, 100)
        
        Vod = Cod * P**1.23
        Vc  = Cv  * P**1.5
        
        df = pd.DataFrame({"P * 10^6":P/10**6, "V_od * 10^6":Vod/10**6, "V_c * 10^6":Vc/10**6})
        
        plot = df.hvplot.line(x='P * 10^6', y=['V_od * 10^6', 'V_c * 10^6'], grid=True, loglog=True, xlim=(10, 5000), ylim=(2, 5000), color=['blue', 'red'], title='Student plot')
        
        
        dP = P_before - P_after
        
        dP_curve = hv.Curve([(P_before/10**6, V_c_before/10**6), (P_after/10**6, V_c_before/10**6)]).opts(color='black', apply_ranges=True, line_dash='solid', line_width=1) * \
                   hv.Text((P_before+P_after)/2/10**6, V_c_before/10**6 + 300, 'dP', fontsize=9)
        
        dV_c_curve = hv.Curve([(P_after/10**6, V_c_before/10**6), (P_after/10**6, (V_c_before-dV_c)/10**6)]).opts(color='black', apply_ranges=True, line_dash='solid', line_width=1) * \
                     hv.Text((P_after/10**6)-50, (V_c_before+V_c_before-dV_c)/2/10**6, 'dVc', fontsize=9)
        
        connection_curve = hv.Curve([[P_before/10**6, V_c_before/10**6], [(P_before-dP)/10**6, (V_c_before-dV_c)/10**6]]).opts(color='black', line_dash='dashed')
        
        a_curves = hv.Curve([[10**-5, (V_c_before-dV_c)/10**6], [P_after/10**6, (V_c_before-dV_c)/10**6]]).opts(color='red', line_dash='dashed') * \
                       hv.Curve([[10**-5, V_c_after/10**6], [P_after/10**6, V_c_after/10**6]]).opts(color='red', line_dash='dashed') * \
                       hv.Curve([[10**2+10, V_c_after/10**6], [10**2+10, (V_c_before-dV_c)/10**6]]).opts(color='black', line_width=1) * \
                       hv.Text(10**2, (V_c_after/10**6+(V_c_before-dV_c)/10**6)/2, text='a', fontsize=9)
        
        b_curves = hv.Curve([[P_after/10**6, V_od_after/10**6], [10**6, V_od_after/10**6]]).opts(color='blue', line_dash='dashed') * \
                       hv.Curve([[P_before/10**6, V_od_before/10**6], [10**6, V_od_before/10**6]]).opts(color='blue', line_dash='dashed') * \
                       hv.Curve([[10**3, V_od_after/10**6], [10**3, (V_od_before)/10**6]]).opts(color='black', line_width=1) * \
                       hv.Text(10**3 + 70, (V_od_after/10**6+V_od_before/10**6)/2, text='b', fontsize=9)
        
        points_old = hv.Points([[P_before/10**6, V_c_before/10**6], [P_before/10**6, V_od_before/10**6]]).opts(size=10, marker='x', color='black', line_color='black')
        points_bet = hv.Points([[P_after/10**6, (V_c_before-dV_c)/10**6]]).opts(size=10, marker='circle', color='black', line_color='black')
        points_new = hv.Points([[P_after/10**6, V_c_after/10**6], [P_after/10**6, V_od_after/10**6]]).opts(size=10, marker='circle', color='white', line_color='black')
        
        points = points_old * points_bet * points_new
        
        student_plot = plot * dP_curve * dV_c_curve * connection_curve * a_curves * b_curves * points

        return student_plot

In [ ]:
### Hidden code block to automatically plot Vod and Vc
def Q4_1(V_c_before, V_od_before, V_c_after, V_od_after, a, b):

    P_before = 600 * 10**6
    P_after = 300 * 10**6
    dV_c = 300 * 10**6
    
    correct_plot, V_c_before_correct, V_c_after_correct, V_od_before_correct, V_od_after_correct, a_correct, b_correct = plot_fig935(P_before, P_after, dV_c)
    student_plot = student_plot_fig935(P_before, P_after, dV_c,
                                        V_c_before,
                                        V_c_after,
                                        V_od_before,
                                        V_od_after,
                                        a,
                                        b)

    incorrect_params = 'The following parameters are wrong:'
    
    if not V_c_before == V_c_before_correct:
        incorrect_params += '\n V_c_before'
    if not V_c_after == V_c_after_correct:
        incorrect_params += '\n V_c_after'
    if not V_od_before == V_od_before_correct:
        incorrect_params += '\n V_od_before'
    if not V_od_after == V_od_after_correct:
        incorrect_params += '\n V_od_after'
    if not a == a_correct:
        incorrect_params += '\n a'
    if not b == b_correct:
        incorrect_params += '\n b'

    if not incorrect_params == 'The following parameters are wrong:':
        incorrect_params += '\n\nRemember to include the 10^6 in your definition of P and V!'
    else:
        print('Your answer is correct!')


    return (correct_plot + student_plot).opts(shared_axes=False)

Q4_1(V_c_before, V_od_before, V_c_after, V_od_after, a, b)

In [ ]:
question_data42 = {
        "name": "Q4-2",
        "question": "Is a larger or smaller than b?",
        "options": {
            "a": "a is larger than b",
            "b": "a is smaller than b",
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data43 = {
        "name": "Q4-3",
        "question": "What does this imply for the longterm coastal evolution?",
        "options": {
            "a": "Accretion updrift of the channel",
            "b": "Erosion updrift of the channel",
            "c": "Accretion downdrift of the channel",
            "d": "Erosion downdrift of the channel"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data42),
          MC(question_data43)
         )

The code below allows you to vary axes type (linear and log-log ), plotting range, coefficients and power. Run the cell below that to replot, and answer the questions below.

In [ ]:
# Choose between 'linear' or 'loglog'                        (default: 'loglog')
axes_type = 'loglog'

# Give your answer in the format [[x1, x2], [y1, y2]]        (default: [[10, 5000], [2, 5000]])
range = [[10, 5000], [2, 5000]]

# Give your answer in the format [C_V, C_od]                 (default: [65 * 10**-6, 65.7 * 10**-4])
coefficients = [65 * 10**-6, 65.7 * 10**-4]

# Powers used for calculation of V_c and V_od respectively.  (default: [1.5, 1.23])
powers = [1.5, 1.23]

In [ ]:
def Q4_4(axes_type, range, coefficients, powers):

    P_before = 600 * 10**6
    P_after = 300 * 10**6
    dV_c = 300 * 10**6
    
    plot, __, __, __, __, __, __ = plot_fig935(P_before, P_after, dV_c, axes_type=axes_type, range=range, coefficients=coefficients, powers=powers, title='')
    
    return plot

Q4_4(axes_type, range, coefficients, powers)

In [ ]:
question_data45 = {
        "name": "Q4-5",
        "question": "What is the advantage of the log-scale?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data46 = {
        "name": "Q4-6",
        "question": "What determines the steepness of the curve?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data47 = {
        "name": "Q4-7",
        "question": "What determines the intercept with the vertical axis?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data48 = {
        "name": "Q4-8",
        "question": "STUART ADD QUESTION HERE",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data45),
          MC(question_data46),
          MC(question_data47),
          MC(question_data48)
         )

Now let's consider colusre 2 from the book (9.35b). The required values are given in the table below. Again, recreate the figure using the values from the table by completing the code below.

|                        | closure 2 |
| :--------------------- | :-------- |
| Prism before           | 600       |
| Prism after            | 300       |
| Difference in Vc <br>  | 470       |

In [ ]:
# Ask students for Vc and P before and directly after intervention. Let them compute it using the Python code. Verify their numbers and plot their answers in the plot.

Vc_before = ...
Vc_after = ...

P_before = ...
P_after = ...

a = ...
b = ...

In [ ]:
# correct answers
V_c_before  = 65*10**-6 * (600*10**6)**1.5
V_c_after   = 60*10**-6 * (300*10**6)**1.5

V_od_before = 65.7*10**-4 * (600*10**6)**1.23
V_od_after  = 64.7*10**-4 * (300*10**6)**1.23


a = 65*10**-6 * (600*10**6)**1.5 - 65*10**-6 * (300*10**6)**1.5 - 470*10**6
b = 65.7*10**-4 * (600*10**6)**1.23 - 65.7*10**-4 * (300*10**6)**1.23

In [ ]:
### Hidden code block to automatically plot Vod and Vc
def Q4_9(V_c_before, V_od_before, V_c_after, V_od_after, a, b):

    P_before = 600 * 10**6
    P_after = 300 * 10**6
    dV_c = 470 * 10**6
    
    correct_plot, V_c_before_correct, V_c_after_correct, V_od_before_correct, V_od_after_correct, a_correct, b_correct = plot_fig935(P_before, P_after, dV_c)
    student_plot = student_plot_fig935(P_before, P_after, dV_c,
                                        V_c_before,
                                        V_c_after,
                                        V_od_before,
                                        V_od_after,
                                        a,
                                        b)

    incorrect_params = 'The following parameters are wrong:'
    
    if not V_c_before == V_c_before_correct:
        incorrect_params += '\n V_c_before'
    if not V_c_after == V_c_after_correct:
        incorrect_params += '\n V_c_after'
    if not V_od_before == V_od_before_correct:
        incorrect_params += '\n V_od_before'
    if not V_od_after == V_od_after_correct:
        incorrect_params += '\n V_od_after'
    if not a == a_correct:
        incorrect_params += '\n a'
    if not b == b_correct:
        incorrect_params += '\n b'

    if not incorrect_params == 'The following parameters are wrong:':
        incorrect_params += '\n\nRemember to include the 10^6 in your definition of P and V!'
        print(incorrect_params)
    else:
        print('Your answer is correct!')


    return (correct_plot + student_plot).opts(shared_axes=False)

Q4_9(V_c_before, V_od_before, V_c_after, V_od_after, a, b)

In [ ]:
# Ask students to compute the new equilibrium values for Vc and Vod using Python code. 
# Verify their numbers. Provide plot function/button to plot these values in the figures too (with the arrows etc. like in the book).

# Vod_before = ...
# Vod_after = ...

# THIS IS ALREADY DONE IN THE BLOCKS ABOVE

Using the plots generated above, answer the following reflective questions.

In [ ]:
question_data410 = {
        "name": "Q4-10",
        "question": "What happens to the outer delta?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data411 = {
        "name": "Q4-11",
        "question": "What happens to channels?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data412 = {
        "name": "Q4-12",
        "question": "What happens to adjacent coast?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data413 = {
        "name": "Q4-13",
        "question": "What is the role of the flats?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data414 = {
        "name": "Q4-14",
        "question": "What is the link to tidal asymmetry?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data415 = {
        "name": "Q4-15",
        "question": "Will there be import or export of sediment?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data410),
          MC(question_data411),
          MC(question_data412),
          MC(question_data413),
          MC(question_data414),
          MC(question_data415)
         )

Let's consider one more closure. Use the numbers from the table below and complete code below. Run the code to generate a figure.
|                        | closure 3 |
| :--------------------- | :-------- |
| Prism before           | 600       |
| Prism after           5| 300       |
| Difference in      r>300 470       |


In [ ]:
V_c_before  = ...
V_c_after   = ...

V_od_before = ...
V_od_after  = ...


a = ...
b = ...

In [ ]:
# correct answers
V_c_before  = 65*10**-6 * (600*10**6)**1.5
V_c_after   = 65*10**-6 * (500*10**6)**1.5

V_od_before = 65.7*10**-4 * (600*10**6)**1.23
V_od_after  = 65.7*10**-4 * (500*10**6)**1.23


a = 65*10**-6 * (600*10**6)**1.5 - 65*10**-6 * (500*10**6)**1.5 - 300*10**6
b = 65.7*10**-4 * (600*10**6)**1.23 - 65.7*10**-4 * (500*10**6)**1.23

In [ ]:
### Hidden code block to automatically plot Vod and Vc
def Q4_16(V_c_before, V_od_before, V_c_after, V_od_after, a, b):

    P_before = 600 * 10**6
    P_after = 500 * 10**6
    dV_c = 300 * 10**6
    
    correct_plot, V_c_before_correct, V_c_after_correct, V_od_before_correct, V_od_after_correct, a_correct, b_correct = plot_fig935(P_before, P_after, dV_c)
    student_plot = student_plot_fig935(P_before, P_after, dV_c,
                                        V_c_before,
                                        V_c_after,
                                        V_od_before,
                                        V_od_after,
                                        a,
                                        b)

    incorrect_params = 'The following parameters are wrong:'
    
    if not V_c_before == V_c_before_correct:
        incorrect_params += '\n V_c_before'
    if not V_c_after == V_c_after_correct:
        incorrect_params += '\n V_c_after'
    if not V_od_before == V_od_before_correct:
        incorrect_params += '\n V_od_before'
    if not V_od_after == V_od_after_correct:
        incorrect_params += '\n V_od_after'
    if not a == a_correct:
        incorrect_params += '\n a'
    if not b == b_correct:
        incorrect_params += '\n b'

    if not incorrect_params == 'The following parameters are wrong:':
        incorrect_params += '\n\nRemember to include the 10^6 in your definition of P and V!'
    else:
        print('Your answer is correct!')


    return (correct_plot + student_plot).opts(shared_axes=False)

Q4_16(V_c_before, V_od_before, V_c_after, V_od_after, a, b)

Using the generated plots, answer the questions below.

In [ ]:
question_data417 = {
        "name": "Q4-17",
        "question": "What happens to the outer delta?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data418 = {
        "name": "Q4-18",
        "question": "What happens to channels?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data419 = {
        "name": "Q4-19",
        "question": "What happens to adjacent coast?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "a",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data420 = {
        "name": "Q4-20",
        "question": "Will there be import or export of sediment?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data417),
          MC(question_data418),
          MC(question_data419),
          MC(question_data420)
         )

Now let us consider a land reclamation. The table below provides the required numbers. Finish the code below to produce a figure.

|                        | accretion |
| :--------------------- | :-------- |
| Prism before           50000       |
| Prism after           250300       |
| Difference in Vc <br>0 470         |

In [ ]:
V_c_before  = ...
V_c_after   = ...

V_od_before = ...
V_od_after  = ...


a = ...
b = ...

In [ ]:
# correct answers
V_c_before  = 65*10**-6 * (500*10**6)**1.5
V_c_after   = 65*10**-6 * (250*10**6)**1.5

V_od_before = 65.7*10**-4 * (500*10**6)**1.23
V_od_after  = 65.7*10**-4 * (250*10**6)**1.23


a = 65*10**-6 * (500*10**6)**1.5 - 65*10**-6 * (250*10**6)**1.5 - 0
b = 65.7*10**-4 * (500*10**6)**1.23 - 65.7*10**-4 * (250*10**6)**1.23

In [ ]:
### Hidden code block to automatically plot Vod and Vc
def Q4_21(V_c_before, V_od_before, V_c_after, V_od_after, a, b):

    P_before = 500 * 10**6
    P_after = 250 * 10**6
    dV_c = 0 * 10**6
    
    correct_plot, V_c_before_correct, V_c_after_correct, V_od_before_correct, V_od_after_correct, a_correct, b_correct = plot_fig935(P_before, P_after, dV_c)
    student_plot = student_plot_fig935(P_before, P_after, dV_c,
                                        V_c_before,
                                        V_c_after,
                                        V_od_before,
                                        V_od_after,
                                        a,
                                        b)

    incorrect_params = 'The following parameters are wrong:'
    
    if not V_c_before == V_c_before_correct:
        incorrect_params += '\n V_c_before'
    if not V_c_after == V_c_after_correct:
        incorrect_params += '\n V_c_after'
    if not V_od_before == V_od_before_correct:
        incorrect_params += '\n V_od_before'
    if not V_od_after == V_od_after_correct:
        incorrect_params += '\n V_od_after'
    if not a == a_correct:
        incorrect_params += '\n a'
    if not b == b_correct:
        incorrect_params += '\n b'

    if not incorrect_params == 'The following parameters are wrong:':
        incorrect_params += '\n\nRemember to include the 10^6 in your definition of P and V!'
    else:
        print('Your answer is correct!')


    return (correct_plot + student_plot).opts(shared_axes=False)

Q4_21(V_c_before, V_od_before, V_c_after, V_od_after, a, b)

Finally, let us consider relative sea level rise. Use the numbers from the table below to finish the code.

|                        | relative sea level rise |
| :--------------------- | :---------------------- |
| Prism before           | 750                     |
| Prism after            | 750                     |
| Difference in Vc <br>  | -200                    |

In [ ]:
Vc_before = ...
Vc_after = ...

P_before = ...
P_after = ...

Vod_before = ...
Vod_after = ...

In [ ]:
# correct answers
V_c_before  = 65*10**-6 * (750*10**6)**1.5
V_c_after   = 65*10**-6 * (750*10**6)**1.5

V_od_before = 65.7*10**-4 * (750*10**6)**1.23
V_od_after  = 65.7*10**-4 * (750*10**6)**1.23


a = 65*10**-6 * (750*10**6)**1.5 - 65*10**-6 * (750*10**6)**1.5 - -200 * 10**6
b = 65.7*10**-4 * (750*10**6)**1.23 - 65.7*10**-4 * (750*10**6)**1.23

In [ ]:
### Hidden code block to automatically plot Vod and Vc
def Q4_22(V_c_before, V_od_before, V_c_after, V_od_after, a, b):

    P_before = 750 * 10**6
    P_after = 750 * 10**6
    dV_c = -200 * 10**6
    
    correct_plot, V_c_before_correct, V_c_after_correct, V_od_before_correct, V_od_after_correct, a_correct, b_correct = plot_fig935(P_before, P_after, dV_c)
    student_plot = student_plot_fig935(P_before, P_after, dV_c,
                                        V_c_before,
                                        V_c_after,
                                        V_od_before,
                                        V_od_after,
                                        a,
                                        b)

    incorrect_params = 'The following parameters are wrong:'
    
    if not V_c_before == V_c_before_correct:
        incorrect_params += '\n V_c_before'
    if not V_c_after == V_c_after_correct:
        incorrect_params += '\n V_c_after'
    if not V_od_before == V_od_before_correct:
        incorrect_params += '\n V_od_before'
    if not V_od_after == V_od_after_correct:
        incorrect_params += '\n V_od_after'
    if not a == a_correct:
        incorrect_params += '\n a'
    if not b == b_correct:
        incorrect_params += '\n b'

    if not incorrect_params == 'The following parameters are wrong:':
        incorrect_params += '\n\nRemember to include the 10^6 in your definition of P and V!'
    else:
        print('Your answer is correct!')


    return (correct_plot + student_plot).opts(shared_axes=False)

Q4_22(V_c_before, V_od_before, V_c_after, V_od_after, a, b)

Finally, answer the question below.

In [ ]:
question_data423 = {
        "name": "Q4-23",
        "question": "Why  does the tidal prism not change as a result of relative sea level rise?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

question_data424 = {
        "name": "Q4-24",
        "question": "The sign (-) of the change in channel volume implies an increase. Why  does relative sea level rise result in an increase of the channel volume?",
        "options": {
            "a": "1STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "b": "2STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "c": "3STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!",
            "d": "4STUART INSERT MULTIPLE OPTIONS HERE, AND INDEX THE CORRECT ONE!"
        },
        "answer": "d",
        "feedback": {"correct": "correct",
                     "incorrect": "incorrect"}
    }

pn.Column(MC(question_data423),
          MC(question_data424)
         )

Wrap-up text (STUART)